# Build The Dataframe

## Import the necessary libraries

In [3]:
import pandas as pd 
import requests

Now use the requests to get the HTML for the Canada postal codes

In [4]:
url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
htmltext = url.text
# print(htmltext)

Using Beautiful Soup it allows use to find the table that contains the 
postal code.

In [5]:
from bs4 import BeautifulSoup
import bs4

soup = bs4.BeautifulSoup(htmltext, 'lxml')
table = soup.find('table','wikitable sortable')
#print (table)

   Now we need to parse the table into columns and store in the dataframe

In [7]:
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text().replace('\n', ''))
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
            print (column_names)
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1

['Postcode', 'Borough', 'Neighbourhood']


Show the first 20 rows

In [8]:
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [9]:
df.describe()

,Postcode,Borough,Neighbourhood
count,289,289,289
unique,180,12,210
top,M9V,Not assigned,Not assigned\n
freq,8,77,78


We need to remove the newline in order to get the proper
column and data

In [10]:
df = df.replace('\n','', regex=True)

Only keep assigned borough names

In [11]:
df = df[df.Borough != "Not assigned"]

In [13]:
df.head(20)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


Now we group by postal code and borough and comma delimit all of the neighborhoos

In [15]:
dfg = df.groupby(['Postcode','Borough'])['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()

In [17]:
dfg.head(10)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [16]:
dfg.shape

(103, 3)